In [1]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from time import time
import pickle
import math
import timm

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize

import warnings
warnings.filterwarnings('ignore')

import random
# Set random seed
SEED = 32
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

from sklearn.metrics import f1_score

# dataset

In [2]:
class cfg:
    data_dir = '/opt/ml/input/data/train'
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'

# 데이터프레임 -> 데이터셋
class CustomDatasetFromDF(Dataset):
    def __init__(self, DataFrame):
        # Transforms
        self.to_tensor = transforms.Compose([
                                             transforms.ToTensor()])
        self.data_info = DataFrame
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.loc[:, 'imageFile'])
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.loc[:, 'allcate'])
        # Third column is for an operation indicator
        # self.operation_arr = np.asarray(self.data_info.iloc[:, 2])
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        img_as_img = Image.open(single_image_name)

        # Check if there is an operation
        # some_operation = self.operation_arr[index]
        # If there is an operation
        # if some_operation:
            # Do some operation on image
            # ...
            # ...
        #    pass
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

In [3]:
with open(f'{cfg.data_dir}/df_0825.pickle', 'rb') as f:
    df = pickle.load(f)

In [4]:
class F1Loss(nn.Module):
    def __init__(self, classes=18, epsilon=1e-7):
        super().__init__()
        self.classes = classes
        self.epsilon = epsilon
    def forward(self, y_pred, y_true):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.classes).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2 * (precision * recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1 - self.epsilon)
        return 1 - f1.mean()

In [4]:
# 5 - fold cross validation
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(df, df['allcate']):
    folds.append((train_idx, valid_idx))

In [5]:
# 동일한 기본값 설정
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 16
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 5 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
# optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# 모델 정의

## ResNet50

In [8]:
# Custom Model Template
class Resnet50(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.resnet50 = models.resnet50(pretrained=True)
        self.resnet50.fc = torch.nn.Linear(in_features=2048, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.resnet50.fc.weight)
        stdv =  1 / math.sqrt(self.resnet50.fc.in_features)
        self.resnet50.fc.bias.data.uniform_(-stdv, stdv)

    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.resnet50(x)
        return x

## EfficientNet b0

In [43]:
# Custom Model Template
class EfficientNet_b0(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('tf_efficientnet_b0', pretrained = True)
        self.model.classifier = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## EfficientNet b1

In [44]:
# Custom Model Template
class EfficientNet_b1(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('tf_efficientnet_b1', pretrained = True)
        self.model.classifier = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## EfficientNet b2

In [45]:
# Custom Model Template
class EfficientNet_b2(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('tf_efficientnet_b2', pretrained = True)
        self.model.classifier = nn.Linear(in_features=1408, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## EfficientNet b3

In [46]:
# Custom Model Template
class EfficientNet_b3(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('tf_efficientnet_b3', pretrained = True)
        self.model.classifier = nn.Linear(in_features=1536, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## EfficientNet b4

In [148]:
# Custom Model Template
class EfficientNet_b4(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('tf_efficientnet_b4', pretrained = True)
        self.model.classifier = nn.Linear(in_features=1792, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## EfficientNet b5

In [48]:
# Custom Model Template
class EfficientNet_b5(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('tf_efficientnet_b5', pretrained = True)
        self.model.classifier = nn.Linear(in_features=2048, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## Vision Transformer

In [49]:
# Custom Model Template
class Vit_large(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('vit_large_patch16_224', pretrained=True)
        self.model.head = nn.Linear(in_features=1024, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.head.weight)
        stdv =  1 / math.sqrt(self.model.head.in_features)
        self.model.head.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

In [50]:
# Custom Model Template
class Vit_base(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('vit_base_patch16_224', pretrained=True)
        self.model.head = nn.Linear(in_features=768, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.head.weight)
        stdv =  1 / math.sqrt(self.model.head.in_features)
        self.model.head.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

In [51]:
# Custom Model Template
class Vit_base_resnet50(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('vit_base_resnet50_384', pretrained=True)
        self.model.head = nn.Linear(in_features=768, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.head.weight)
        stdv =  1 / math.sqrt(self.model.head.in_features)
        self.model.head.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

## EfficientNet pruned

In [48]:
# Custom Model Template
class efficientnet_b1_pruned(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('efficientnet_b1_pruned', pretrained=True)
        self.model.classifier = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

In [53]:
# Custom Model Template
class efficientnet_b2_pruned(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('efficientnet_b2_pruned', pretrained=True)
        self.model.classifier = nn.Linear(in_features=1408, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

In [54]:
# Custom Model Template
class efficientnet_b3_pruned(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        """
        1. 위와 같이 생성자의 parameter 에 num_claases 를 포함해주세요.
        2. 나만의 모델 아키텍쳐를 디자인 해봅니다.
        3. 모델의 output_dimension 은 num_classes 로 설정해주세요.
        """

        self.model = timm.create_model('efficientnet_b3_pruned', pretrained=True)
        self.model.classifier = nn.Linear(in_features=1536, out_features=num_classes, bias=True)
        nn.init.xavier_uniform_(self.model.classifier.weight)
        stdv =  1 / math.sqrt(self.model.classifier.in_features)
        self.model.classifier.bias.data.uniform_(-stdv, stdv)


    def forward(self, x):
        """
        1. 위에서 정의한 모델 아키텍쳐를 forward propagation 을 진행해주세요
        2. 결과로 나온 output 을 return 해주세요
        """
        x = self.model(x)
        return x

# 학습

In [216]:
def training(Model):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    CLASS_NUM = 18
    BATCH_SIZE = 16
    LEARNING_RATE = 0.0001
    NUM_EPOCH = 5
    
    target_model = Model
    target_model.to(device)

    train_data = CustomDatasetFromDF(df.iloc[fold[0]])
    valid_data = CustomDatasetFromDF(df.iloc[fold[1]])

    # Mnist Dataset을 DataLoader에 붙이기
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE)

    dataloaders = {
        "train" : train_dataloader,
        "test" : valid_dataloader
    }
    ### 학습 코드 시작
    best_test_accuracy = 0.
    best_test_f1score = 0.
    best_test_loss = 1.


    for epoch in range(NUM_EPOCH):
      for phase in ["train", "test"]:
        running_loss = 0.
        running_acc = 0.
        running_f1 = 0.
        if phase == "train":
          target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "test":
          target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
          # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
          # hint - with, pbar
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

          with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            logits = target_model(images)
            _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
            loss = loss_fn(logits, labels)

            y_true = F.one_hot(labels, 18).to(torch.float32)
            y_pred = logits

            tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
            tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
            fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
            fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

            precision = tp / (tp + fp + 1e-7)
            recall = tp / (tp + fn + 1e-7)

            f1 = 2 * (precision * recall) / (precision + recall + 1e-7)
            f1 = f1.clamp(min=1e-7, max=1 - 1e-7)

            if phase == "train":
              loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
              optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

          running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
          running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
          running_f1 += f1.mean()



        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        epoch_f1 = running_f1 / len(dataloaders[phase])

        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 loss : {epoch_loss:.3f}, 평균 f1 Score: {epoch_f1:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
          print('Get best model!')
          best_test_loss = epoch_loss
          best_test_accuracy = epoch_acc
          best_test_f1score = epoch_f1
          best_model = target_model

    print("학습 종료!")
    print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}, 최고 f1 score : {best_test_f1score}")

    return best_model

In [217]:
# efficientnet_b4, 5-fold ensemble 용
def training_b4():
    Models = {}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    CLASS_NUM = 18
    BATCH_SIZE = 16
    LEARNING_RATE = 0.0001
    NUM_EPOCH = 5
    
    for idx, fold in enumerate(folds):
        target_model = timm.create_model('tf_efficientnet_b4', pretrained = True)
        target_model.classifier = nn.Linear(in_features=1792, out_features=CLASS_NUM, bias=True)
        nn.init.xavier_uniform_(target_model.classifier.weight)
        stdv =  1 / math.sqrt(target_model.classifier.in_features)
        target_model.classifier.bias.data.uniform_(-stdv, stdv)
        target_model.to(device)
        
        train_data = CustomDatasetFromDF(df.iloc[fold[0]])
        valid_data = CustomDatasetFromDF(df.iloc[fold[1]])

        # Mnist Dataset을 DataLoader에 붙이기
        train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
        valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

        loss_fn = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE)

        dataloaders = {
            "train" : train_dataloader,
            "test" : valid_dataloader
        }
        ### 학습 코드 시작
        best_test_accuracy = 0.
        best_test_f1score = 0.
        best_test_loss = 1.


        for epoch in range(NUM_EPOCH):
          for phase in ["train", "test"]:
            running_loss = 0.
            running_acc = 0.
            running_f1 = 0.
            if phase == "train":
              target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
            elif phase == "test":
              target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

            for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
              # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
              # hint - with, pbar
              images = images.to(device)
              labels = labels.to(device)

              optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

              with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
                logits = target_model(images)
                _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
                loss = loss_fn(logits, labels)

                y_true = F.one_hot(labels, 18).to(torch.float32)
                y_pred = logits

                tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
                tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
                fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
                fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

                precision = tp / (tp + fp + 1e-7)
                recall = tp / (tp + fn + 1e-7)

                f1 = 2 * (precision * recall) / (precision + recall + 1e-7)
                f1 = f1.clamp(min=1e-7, max=1 - 1e-7)

                if phase == "train":
                  loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
                  optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

              running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
              running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
              running_f1 += f1.mean()



            # 한 epoch이 모두 종료되었을 때,
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_acc / len(dataloaders[phase].dataset)
            epoch_f1 = running_f1 / len(dataloaders[phase])

            print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 loss : {epoch_loss:.3f}, 평균 f1 Score: {epoch_f1:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
            if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
              print('Get best model!')
              best_test_loss = epoch_loss
              best_test_accuracy = epoch_acc
              best_test_f1score = epoch_f1
              best_model = target_model

        print("학습 종료!")
        print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}, 최고 f1 score : {best_test_f1score}")
        Models[idx] = best_model
    return Models

## ResNet

In [10]:
training(Resnet50(18))

KeyboardInterrupt: 

## EfficientNet

In [72]:
torch.cuda.empty_cache()
training(EfficientNet_b0(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.573, 평균 f1 Score: 0.012, 평균 Accuracy : 0.829



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.196, 평균 f1 Score: 0.011, 평균 Accuracy : 0.933
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.160, 평균 f1 Score: 0.013, 평균 Accuracy : 0.948



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.117, 평균 f1 Score: 0.011, 평균 Accuracy : 0.957
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.070, 평균 f1 Score: 0.013, 평균 Accuracy : 0.980



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.075, 평균 f1 Score: 0.011, 평균 Accuracy : 0.975
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.040, 평균 f1 Score: 0.012, 평균 Accuracy : 0.988



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.060, 평균 f1 Score: 0.011, 평균 Accuracy : 0.981
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.029, 평균 f1 Score: 0.012, 평균 Accuracy : 0.992



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.059, 평균 f1 Score: 0.011, 평균 Accuracy : 0.983
Get best model!
학습 종료!
최고 accuracy : 0.9828042387962341, 최고 낮은 loss : 0.05867854012210657, 최고 f1 score : 0.01099776104092598


In [59]:
torch.cuda.empty_cache()
training(EfficientNet_b1(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.521, 평균 f1 Score: 0.202, 평균 Accuracy : 0.843



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.172, 평균 f1 Score: 0.183, 평균 Accuracy : 0.938
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.143, 평균 f1 Score: 0.205, 평균 Accuracy : 0.954



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.094, 평균 f1 Score: 0.188, 평균 Accuracy : 0.969
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.059, 평균 f1 Score: 0.206, 평균 Accuracy : 0.981



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.088, 평균 f1 Score: 0.178, 평균 Accuracy : 0.972
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.041, 평균 f1 Score: 0.205, 평균 Accuracy : 0.987



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.095, 평균 f1 Score: 0.174, 평균 Accuracy : 0.970



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.033, 평균 f1 Score: 0.205, 평균 Accuracy : 0.990



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.072, 평균 f1 Score: 0.175, 평균 Accuracy : 0.980
Get best model!
학습 종료!
최고 accuracy : 0.9804232716560364, 최고 낮은 loss : 0.07173065482087497, 최고 f1 score : 0.17497272789478302


In [60]:
torch.cuda.empty_cache()
training(EfficientNet_b2(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.537, 평균 f1 Score: 0.214, 평균 Accuracy : 0.838



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.181, 평균 f1 Score: 0.204, 평균 Accuracy : 0.940
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.126, 평균 f1 Score: 0.205, 평균 Accuracy : 0.960



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.111, 평균 f1 Score: 0.198, 평균 Accuracy : 0.962
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.058, 평균 f1 Score: 0.207, 평균 Accuracy : 0.982



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.078, 평균 f1 Score: 0.193, 평균 Accuracy : 0.976
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.039, 평균 f1 Score: 0.206, 평균 Accuracy : 0.989



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.081, 평균 f1 Score: 0.184, 평균 Accuracy : 0.979



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.032, 평균 f1 Score: 0.205, 평균 Accuracy : 0.990



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.095, 평균 f1 Score: 0.190, 평균 Accuracy : 0.976
학습 종료!
최고 accuracy : 0.9759259223937988, 최고 낮은 loss : 0.07794957169622341, 최고 f1 score : 0.19325381517410278


In [57]:
torch.cuda.empty_cache()
training(EfficientNet_b3(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.494, 평균 f1 Score: 0.201, 평균 Accuracy : 0.849



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.170, 평균 f1 Score: 0.206, 평균 Accuracy : 0.947
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.115, 평균 f1 Score: 0.206, 평균 Accuracy : 0.964



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.093, 평균 f1 Score: 0.198, 평균 Accuracy : 0.973
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.053, 평균 f1 Score: 0.205, 평균 Accuracy : 0.985



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.077, 평균 f1 Score: 0.196, 평균 Accuracy : 0.977
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.039, 평균 f1 Score: 0.207, 평균 Accuracy : 0.987



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.060, 평균 f1 Score: 0.190, 평균 Accuracy : 0.983
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.038, 평균 f1 Score: 0.205, 평균 Accuracy : 0.989



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.071, 평균 f1 Score: 0.195, 평균 Accuracy : 0.980
학습 종료!
최고 accuracy : 0.9828042387962341, 최고 낮은 loss : 0.06035067331619935, 최고 f1 score : 0.18976990878582


In [218]:
torch.cuda.empty_cache()
model_b4 = training_b4()


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.402, 평균 f1 Score: 0.216, 평균 Accuracy : 0.884



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.094, 평균 f1 Score: 0.214, 평균 Accuracy : 0.970
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.077, 평균 f1 Score: 0.205, 평균 Accuracy : 0.977



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.123, 평균 f1 Score: 0.213, 평균 Accuracy : 0.962



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.037, 평균 f1 Score: 0.207, 평균 Accuracy : 0.989


...


현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.044, 평균 f1 Score: 0.219, 평균 Accuracy : 0.987
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.025, 평균 f1 Score: 0.204, 평균 Accuracy : 0.993



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.035, 평균 f1 Score: 0.211, 평균 Accuracy : 0.989
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.022, 평균 f1 Score: 0.192, 평균 Accuracy : 0.994



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.035, 평균 f1 Score: 0.208, 평균 Accuracy : 0.989
학습 종료!
최고 accuracy : 0.9894180297851562, 최고 낮은 loss : 0.034780217180123724, 최고 f1 score : 0.21110902726650238


In [61]:
torch.cuda.empty_cache()
training(EfficientNet_b5(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.512, 평균 f1 Score: 0.207, 평균 Accuracy : 0.843



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.173, 평균 f1 Score: 0.203, 평균 Accuracy : 0.938
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.122, 평균 f1 Score: 0.206, 평균 Accuracy : 0.960



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.084, 평균 f1 Score: 0.198, 평균 Accuracy : 0.974
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.051, 평균 f1 Score: 0.205, 평균 Accuracy : 0.984



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.145, 평균 f1 Score: 0.196, 평균 Accuracy : 0.959



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.047, 평균 f1 Score: 0.205, 평균 Accuracy : 0.986



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.086, 평균 f1 Score: 0.197, 평균 Accuracy : 0.973



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.040, 평균 f1 Score: 0.207, 평균 Accuracy : 0.988



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.050, 평균 f1 Score: 0.196, 평균 Accuracy : 0.987
Get best model!
학습 종료!
최고 accuracy : 0.9873015880584717, 최고 낮은 loss : 0.0498379780537718, 최고 f1 score : 0.19564712047576904


## vit

In [86]:
torch.cuda.empty_cache()
training(Vit_large(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.545, 평균 f1 Score: 0.271, 평균 Accuracy : 0.830



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.295, 평균 f1 Score: 0.203, 평균 Accuracy : 0.897
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.275, 평균 f1 Score: 0.297, 평균 Accuracy : 0.910



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.321, 평균 f1 Score: 0.203, 평균 Accuracy : 0.893



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.162, 평균 f1 Score: 0.306, 평균 Accuracy : 0.950



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.427, 평균 f1 Score: 0.213, 평균 Accuracy : 0.859
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.170, 평균 f1 Score: 0.300, 평균 Accuracy : 0.945



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.214, 평균 f1 Score: 0.214, 평균 Accuracy : 0.933
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.109, 평균 f1 Score: 0.303, 평균 Accuracy : 0.967



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.163, 평균 f1 Score: 0.206, 평균 Accuracy : 0.951
학습 종료!
최고 accuracy : 0.9513227939605713, 최고 낮은 loss : 0.16282613948973082, 최고 f1 score : 0.0028347852639853954


In [87]:
torch.cuda.empty_cache()
training(Vit_base(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 1.203, 평균 f1 Score: 0.213, 평균 Accuracy : 0.631



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.378, 평균 f1 Score: 0.198, 평균 Accuracy : 0.881
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.322, 평균 f1 Score: 0.280, 평균 Accuracy : 0.895



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.318, 평균 f1 Score: 0.204, 평균 Accuracy : 0.895
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.229, 평균 f1 Score: 0.283, 평균 Accuracy : 0.923



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.235, 평균 f1 Score: 0.196, 평균 Accuracy : 0.922



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.175, 평균 f1 Score: 0.293, 평균 Accuracy : 0.941



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.242, 평균 f1 Score: 0.209, 평균 Accuracy : 0.927
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.148, 평균 f1 Score: 0.297, 평균 Accuracy : 0.952



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.225, 평균 f1 Score: 0.201, 평균 Accuracy : 0.929
학습 종료!
최고 accuracy : 0.9285714626312256, 최고 낮은 loss : 0.22508089318428012, 최고 f1 score : 0.0028347852639853954


In [89]:
torch.cuda.empty_cache()
training(Vit_base_resnet50(18))

RuntimeError: The size of tensor a (197) must match the size of tensor b (577) at non-singleton dimension 1

## efficientnet_pruned

In [51]:
torch.cuda.empty_cache()
model_b1_pruned = training(efficientnet_b1_pruned(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.481, 평균 f1 Score: 0.219, 평균 Accuracy : 0.862



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.202, 평균 f1 Score: 0.197, 평균 Accuracy : 0.941
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.123, 평균 f1 Score: 0.207, 평균 Accuracy : 0.960



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.105, 평균 f1 Score: 0.196, 평균 Accuracy : 0.964
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.049, 평균 f1 Score: 0.205, 평균 Accuracy : 0.986



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.076, 평균 f1 Score: 0.192, 평균 Accuracy : 0.972
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.033, 평균 f1 Score: 0.205, 평균 Accuracy : 0.991



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.049, 평균 f1 Score: 0.186, 평균 Accuracy : 0.985
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.030, 평균 f1 Score: 0.197, 평균 Accuracy : 0.991



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.083, 평균 f1 Score: 0.182, 평균 Accuracy : 0.976
학습 종료!
최고 accuracy : 0.9846560955047607, 최고 낮은 loss : 0.04934669828984644, 최고 f1 score : 0.18570572137832642


In [63]:
torch.cuda.empty_cache()
training(efficientnet_b2_pruned(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.522, 평균 f1 Score: 0.202, 평균 Accuracy : 0.840



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.190, 평균 f1 Score: 0.197, 평균 Accuracy : 0.936
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.139, 평균 f1 Score: 0.202, 평균 Accuracy : 0.955



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.125, 평균 f1 Score: 0.192, 평균 Accuracy : 0.962
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.066, 평균 f1 Score: 0.203, 평균 Accuracy : 0.979



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.083, 평균 f1 Score: 0.188, 평균 Accuracy : 0.975
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.042, 평균 f1 Score: 0.196, 평균 Accuracy : 0.987



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.090, 평균 f1 Score: 0.187, 평균 Accuracy : 0.971



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.036, 평균 f1 Score: 0.198, 평균 Accuracy : 0.989



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.071, 평균 f1 Score: 0.186, 평균 Accuracy : 0.980
Get best model!
학습 종료!
최고 accuracy : 0.97989422082901, 최고 낮은 loss : 0.07068138973713975, 최고 f1 score : 0.186245858669281


In [64]:
torch.cuda.empty_cache()
training(efficientnet_b3_pruned(18))


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.463, 평균 f1 Score: 0.227, 평균 Accuracy : 0.854



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.188, 평균 f1 Score: 0.211, 평균 Accuracy : 0.932
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.158, 평균 f1 Score: 0.207, 평균 Accuracy : 0.947



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.132, 평균 f1 Score: 0.204, 평균 Accuracy : 0.956
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.084, 평균 f1 Score: 0.206, 평균 Accuracy : 0.972



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.111, 평균 f1 Score: 0.211, 평균 Accuracy : 0.966
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.058, 평균 f1 Score: 0.208, 평균 Accuracy : 0.981



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.089, 평균 f1 Score: 0.203, 평균 Accuracy : 0.970
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.048, 평균 f1 Score: 0.207, 평균 Accuracy : 0.984



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.067, 평균 f1 Score: 0.199, 평균 Accuracy : 0.982
Get best model!
학습 종료!
최고 accuracy : 0.982275128364563, 최고 낮은 loss : 0.06684485905127423, 최고 f1 score : 0.1985614001750946


# 모델 비교 실험

In [29]:
target_model = Resnet50(18)

In [30]:
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [31]:
## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 5 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

cuda is using!


In [38]:
### 학습 코드 시작
best_test_accuracy = 0.
best_test_f1score = 0.
best_test_loss = 1.


for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    running_f1 = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)
        
        y_true = F.one_hot(labels, 18).to(torch.float32)
        y_pred = logits

        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + 1e-7)
        recall = tp / (tp + fn + 1e-7)

        f1 = 2 * (precision * recall) / (precision + recall + 1e-7)
        f1 = f1.clamp(min=1e-7, max=1 - 1e-7)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
      running_f1 += f1.mean()
      
      

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    epoch_f1 = running_f1 / NUM_EPOCH

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 loss : {epoch_loss:.3f}, 평균 f1 Score: {epoch_f1:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_f1score < epoch_f1:
      print('Get best model!')
      best_test_f1 = epoch_f1
      best_model = target_model
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}, 최고 f1 score : {best_test_f1}")


현재 epoch-0의 train-데이터 셋에서 평균 loss : 0.012, 평균 f1 Score: 0.003, 평균 Accuracy : 0.997



현재 epoch-0의 test-데이터 셋에서 평균 loss : 0.050, 평균 f1 Score: 0.003, 평균 Accuracy : 0.985
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 loss : 0.018, 평균 f1 Score: 0.003, 평균 Accuracy : 0.995



현재 epoch-1의 test-데이터 셋에서 평균 loss : 0.221, 평균 f1 Score: 0.003, 평균 Accuracy : 0.945
Get best model!



현재 epoch-2의 train-데이터 셋에서 평균 loss : 0.018, 평균 f1 Score: 0.003, 평균 Accuracy : 0.994



현재 epoch-2의 test-데이터 셋에서 평균 loss : 0.058, 평균 f1 Score: 0.003, 평균 Accuracy : 0.983
Get best model!



현재 epoch-3의 train-데이터 셋에서 평균 loss : 0.013, 평균 f1 Score: 0.003, 평균 Accuracy : 0.996



현재 epoch-3의 test-데이터 셋에서 평균 loss : 0.054, 평균 f1 Score: 0.003, 평균 Accuracy : 0.985
Get best model!



현재 epoch-4의 train-데이터 셋에서 평균 loss : 0.007, 평균 f1 Score: 0.003, 평균 Accuracy : 0.998



현재 epoch-4의 test-데이터 셋에서 평균 loss : 0.046, 평균 f1 Score: 0.003, 평균 Accuracy : 0.986
Get best model!
학습 종료!
최고 accuracy : 0.985714316368103, 최고 낮은 loss : 0.04594681285078199, 최고 f1 score : 0.0028347852639853954


In [ ]:
def training():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    target_model = models.resnet50(pretrained=True)
    target_model.to(device)
    CLASS_NUM = 18
    target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
    nn.init.xavier_uniform_(target_model.fc.weight)
    stdv =  1 / math.sqrt(target_model.fc.in_features)
    target_model.fc.bias.data.uniform_(-stdv, stdv)
    
    train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
    valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

    # Mnist Dataset을 DataLoader에 붙이기
    BATCH_SIZE = 64
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    LEARNING_RATE = 0.0001
    NUM_EPOCH = 5
    loss_fn = F1Loss()
    optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE)

    dataloaders = {
        "train" : train_dataloader,
        "valid" : valid_dataloader
    }
    ### 학습 코드 시작
    best_test_accuracy = 0.
    best_test_loss = 1.


    for epoch in range(NUM_EPOCH):
      for phase in ["train", "valid"]:
        running_loss = 0.
        running_acc = 0.
        if phase == "train":
          target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
        elif phase == "valid":
          target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
          # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
          # hint - with, pbar
          images = images.to(device)
          labels = labels.to(device)

          optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

          with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            logits = target_model(images)
            _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
            loss = loss_fn(logits, labels)

            if phase == "train":
              loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
              optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

          running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
          running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

        # 한 epoch이 모두 종료되었을 때,
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)

        print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 F1 score : {1 - epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
        if phase == "valid" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
          best_test_accuracy = epoch_acc
        if phase == "valid" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
          print('Get best model!')
          best_test_loss = epoch_loss
          best_model = target_model

    print("학습 종료!")
    print(f"최고 accuracy : {best_test_accuracy}, 최고 높은 F1 score : {1 - best_test_loss}")

# XGBoost 학습

In [11]:
# 모델의 fully connected layer 제거
# 이미지의 특성 매트릭스 추출을 위함
class Resnet50_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet50 = models.resnet50(pretrained=True)

    def forward(self, x):
        x = self.resnet50.conv1(x)
        x = self.resnet50.bn1(x)
        x = self.resnet50.relu(x)
        x = self.resnet50.maxpool(x)
        x = self.resnet50.layer1(x)
        x = self.resnet50.layer2(x)
        x = self.resnet50.layer3(x)
        x = self.resnet50.layer4(x)
        x = self.resnet50.avgpool(x)
        return x

In [19]:
device = "cuda" if torch.cuda.is_available() else "cpu"

xgm = Resnet50_for_xgboost().to(device)

for params in xgm.parameters():
    params.requires_grad = False

# 동일한 기본값 설정
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

BATCH_SIZE = 16
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                                               num_workers=4, drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False,
                                               num_workers=4, drop_last=True)

dataloaders = {"train" : train_dataloader, "valid" : valid_dataloader}

intermediate_output = []
labels_output = []

xgm.eval()
for ind, (images, labels) in enumerate(tqdm(dataloaders["train"])):
    images = images.to(device)
    labels = labels.to(device)
    intermediate_output.append(xgm(images).squeeze())
    labels_output.append(labels)

In [27]:
train_X = torch.cat(intermediate_output).cpu().numpy()
train_X

array([[0.21668796, 0.37885714, 0.9215023 , ..., 0.11622887, 0.10953175,
        0.33929878],
       [0.20218973, 0.4073317 , 1.3194582 , ..., 0.07869539, 0.29567274,
        0.85543364],
       [0.24691093, 0.43906817, 1.0445836 , ..., 0.1949091 , 0.35603246,
        0.4635967 ],
       ...,
       [0.07526978, 0.2699431 , 0.81679744, ..., 0.2250255 , 0.26572612,
        0.9031057 ],
       [0.14857924, 0.60255784, 1.1023167 , ..., 0.24992454, 0.29177183,
        0.4671427 ],
       [0.2558588 , 0.37128514, 1.0838196 , ..., 0.1774542 , 0.09716096,
        0.73797363]], dtype=float32)

In [28]:
train_y = torch.cat(labels_output).cpu().numpy()

In [32]:
# XGBoost 모델 학습
from xgboost import XGBClassifier

xgbmodel = XGBClassifier(objective='multi:softprob', 
                         num_class= 18)
xgbmodel.fit(train_X, train_y)

[02:04:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_class=18, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"

xgm = Resnet50_for_xgboost().to(device)

for params in xgm.parameters():
    params.requires_grad = False

# 동일한 기본값 설정
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

BATCH_SIZE = 16
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                                               num_workers=4, drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False,
                                               num_workers=4, drop_last=True)

dataloaders = {"train" : train_dataloader, "valid" : valid_dataloader}

intermediate_output = []
labels_output = []

xgm.eval()
for ind, (images, labels) in enumerate(tqdm(dataloaders["valid"])):
    images = images.to(device)
    labels = labels.to(device)
    intermediate_output.append(xgm(images).squeeze())
    labels_output.append(labels)

In [34]:
valid_X = torch.cat(intermediate_output).cpu().numpy()

In [35]:
valid_y = torch.cat(labels_output).cpu().numpy()

In [36]:
xgbmodel.score(valid_X, valid_y)

0.840042372881356

In [40]:
y_pred = xgbmodel.predict(valid_X)

In [42]:
y_pred.shape

(3776,)

In [43]:
valid_y.shape

(3776,)

In [46]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

f1_score(valid_y, y_pred, average='macro')

0.6656326308202489

In [47]:
from sklearn.metrics import accuracy_score

accuracy_score(valid_y, y_pred)

0.840042372881356

## 학습된 efficientnet b1 prunded 모델로 이미지 특성 추출

In [62]:
# Custom Model Template
class b1_pruned_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b1_pruned = model_b1_pruned.model
        
    def forward(self, x):
        x = self.b1_pruned.conv_stem(x)
        x = self.b1_pruned.bn1(x)
        x = self.b1_pruned.act1(x)
        x = self.b1_pruned.blocks(x)
        x = self.b1_pruned.conv_head(x)
        x = self.b1_pruned.bn2(x)
        x = self.b1_pruned.act2(x)
        x = self.b1_pruned.global_pool(x)
        return x

In [84]:
device = "cuda" if torch.cuda.is_available() else "cpu"
xgm = b1_pruned_for_xgboost().to(device)

for params in xgm.parameters():
    params.requires_grad = False

# 동일한 기본값 설정
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

BATCH_SIZE = 16
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                                               num_workers=4, drop_last=True)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False,
                                               num_workers=4, drop_last=True)

dataloaders = {"train" : train_dataloader, "valid" : valid_dataloader}

from collections import defaultdict

xgb_data = defaultdict(list)
xgm.eval()
phases = ["train", "valid"]
for phase in phases:
    intermediate_output = []
    labels_output = []
    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
        images = images.to(device)
        labels = labels.to(device)
        intermediate_output.append(xgm(images).squeeze())
        labels_output.append(labels)
    xgb_data[phase].append(torch.cat(intermediate_output).cpu().numpy())
    xgb_data[phase].append(torch.cat(labels_output).cpu().numpy())

In [122]:
from xgboost import XGBClassifier

xgbmodel_b1_pruned = XGBClassifier(objective='multi:softprob', num_class= 18)
xgbmodel_b1_pruned.fit(xgb_data['train'][0], xgb_data['train'][1])

[04:14:51] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_class=18, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [123]:
xgb_data['train'][0].shape

(15120, 1280)

In [124]:
y_true = xgb_data['valid'][1]
y_pred = xgbmodel_b1_pruned.predict(xgb_data['valid'][0])

In [125]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print(f'Accuracy: {accuracy_score(y_true, y_pred)}, f1_score: {f1_score(y_true, y_pred, average="macro")}')

Accuracy: 0.9796080508474576, f1_score: 0.952992028678503


In [87]:
xgb_data['valid'][0].shape

(3776, 1280)

In [120]:
y_true

array([4, 4, 4, ..., 6, 0, 0])

In [121]:
y_pred

array([4, 4, 4, ..., 6, 0, 0])

In [135]:
train_X = np.concatenate([xgb_data['train'][0], xgb_data['valid'][0]])
train_y = np.concatenate([xgb_data['train'][1], xgb_data['valid'][1]])

train_X.shape, train_y.shape

((18896, 1280), (18896,))

In [136]:
xgbmodel_b1_pruned_all = XGBClassifier(objective='mlogloss', num_class= 18)
xgbmodel_b1_pruned_all.fit(train_X, train_y)

[04:36:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_class=18, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## PCA 변환, 차원 축소

In [139]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(xgb_data['train'][0])

In [141]:
X_reduced.shape

(15120, 195)

In [143]:
xgbmodel_b1_pruned_reduced = XGBClassifier(objective='mlogloss', num_class= 18)
xgbmodel_b1_pruned_reduced.fit(X_reduced, xgb_data['train'][1])

[04:44:16] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_class=18, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [147]:
X_valid_reduced = pca.transform(xgb_data['valid'][0])
y_pred_reduced = xgbmodel_b1_pruned_reduced.predict(X_valid_reduced)

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print(f'Accuracy: {accuracy_score(y_true, y_pred_reduced)}, f1_score: {f1_score(y_true, y_pred_reduced, average="macro")}')

Accuracy: 0.9753707627118644, f1_score: 0.9529174884338902


## 제출 파일 만들기

In [90]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

In [91]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [92]:
class TestDataset(Dataset):
    def __init__(self, img_paths):
        self.img_paths = img_paths
        self.transform = transforms.Compose([
                                             transforms.ToTensor()])

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [103]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
dataset = TestDataset(image_paths)

loader = DataLoader(
    dataset,
    shuffle=False,
    batch_size=100
)

In [104]:
len(loader)

126

In [105]:
test_data = []
intermediate_output = []
for ind, images in enumerate(tqdm(loader)):
    images = images.to(device)
    intermediate_output.append(xgm(images).squeeze())
test_data.append(torch.cat(intermediate_output).cpu().numpy())

In [130]:
test_data[0]

array([[ 0.30339062,  1.4873213 ,  0.37995267, ...,  0.20099774,
         0.16666704,  1.1302907 ],
       [ 0.04240502,  0.015024  ,  0.51149863, ...,  0.5982672 ,
         0.78917205,  0.43750882],
       [ 0.13172886,  1.7017627 ,  0.35769626, ...,  0.51622677,
         0.01860787,  1.3759849 ],
       ...,
       [ 0.07651859, -0.14223807,  0.41448256, ...,  0.07820901,
        -0.12151265, -0.04866683],
       [ 0.48232627, -0.04635229,  0.66566   , ...,  0.8424868 ,
         0.7519783 ,  0.3553482 ],
       [-0.0841377 ,  0.06739157,  1.8308672 , ...,  0.11808705,
         0.3289736 , -0.11402351]], dtype=float32)

In [106]:
test_data[0].shape

(12600, 1280)

In [126]:
y_pred = xgbmodel_b1_pruned.predict(test_data[0])

In [127]:
y_pred

array([13,  1, 13, ...,  9,  1,  7])

In [128]:
submission['ans'] = y_pred

In [ ]:
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

In [129]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [137]:
y_pred_all = xgbmodel_b1_pruned_all.predict(test_data[0])

In [138]:
y_pred_all

array([13,  1, 13, ...,  9,  1,  7])

In [114]:
submission.to_csv(os.path.join(test_dir, 'b1_pruned_xgb.csv'), index=False)
print('test inference is done!')

test inference is done!


## 학습된 b4 모델에 적용

In [151]:
# Custom Model Template
class b4_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b4 = model_b4.model
        
    def forward(self, x):
        x = self.b4.conv_stem(x)
        x = self.b4.bn1(x)
        x = self.b4.act1(x)
        x = self.b4.blocks(x)
        x = self.b4.conv_head(x)
        x = self.b4.bn2(x)
        x = self.b4.act2(x)
        x = self.b4.global_pool(x)
        return x

In [187]:
device = "cuda" if torch.cuda.is_available() else "cpu"
xgm = b4_for_xgboost().to(device)

for params in xgm.parameters():
    params.requires_grad = False

# 동일한 기본값 설정
data_dict = {}
for idx, fold in tqdm(enumerate(folds)):
    train_data = CustomDatasetFromDF(df.iloc[fold[0]])
    valid_data = CustomDatasetFromDF(df.iloc[fold[1]])

    BATCH_SIZE = 16
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                                                   num_workers=4, drop_last=True)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False,
                                                   num_workers=4, drop_last=True)

    dataloaders = {"train" : train_dataloader, "valid" : valid_dataloader}

    from collections import defaultdict

    xgb_data = defaultdict(list)
    xgm.eval()
    phases = ["train", "valid"]
    for phase in phases:
        intermediate_output = []
        labels_output = []
        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
            images = images.to(device)
            labels = labels.to(device)
            intermediate_output.append(xgm(images).squeeze())
            labels_output.append(labels)
        xgb_data[phase].append(torch.cat(intermediate_output).cpu().numpy())
        xgb_data[phase].append(torch.cat(labels_output).cpu().numpy())
    
    data_dict[idx] = xgb_data

In [153]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(xgb_data['train'][0])

In [314]:
data_dict[0]['train'][0].shape, data_dict[0]['train'][1].shape

((15120, 1792), (15120,))

In [316]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_pred = xgbmodel_b4.predict(xgb_data['valid'][0])
y_true = xgb_data['valid'][1]

print(f'Accuracy: {accuracy_score(y_true, y_pred)}, f1_score: {f1_score(y_true, y_pred, average="macro")}')

Accuracy: 0.9891419491525424, f1_score: 0.9707808189119498


In [312]:
xgbmodel_b4 = XGBClassifier(objective='mlogloss', num_class= 18)
xgbmodel_b4.fit(data_dict[0]['train'][0], data_dict[0]['train'][1])

[01:53:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_class=18, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [307]:
xgb_data['train'][0].shape

(15120, 1792)

In [315]:
X_reduced.shape, X_valid_reduced.shape

((15120, 132), (3776, 132))

In [156]:
xgbmodel_b4_reduced = XGBClassifier(objective='mlogloss', num_class= 18)
xgbmodel_b4_reduced.fit(X_reduced, xgb_data['train'][1])

[06:34:17] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_class=18, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [158]:
X_valid_reduced = pca.transform(xgb_data['valid'][0])
y_pred_reduced = xgbmodel_b4_reduced.predict(X_valid_reduced)
y_true = xgb_data['valid'][1]

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print(f'Accuracy: {accuracy_score(y_true, y_pred_reduced)}, f1_score: {f1_score(y_true, y_pred_reduced, average="macro")}')

Accuracy: 0.988082627118644, f1_score: 0.9719054650719767


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
xgm = b1_pruned_for_xgboost().to(device)

for params in xgm.parameters():
    params.requires_grad = False

# 동일한 기본값 설정
fold_list = []
for fold in folds:
    train_data = CustomDatasetFromDF(df.iloc[fold[0]])
    valid_data = CustomDatasetFromDF(df.iloc[fold[1]])

    BATCH_SIZE = 16
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                                                   num_workers=4, drop_last=True)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False,
                                                   num_workers=4, drop_last=True)

    dataloaders = {"train" : train_dataloader, "valid" : valid_dataloader}

    from collections import defaultdict

    xgb_data = defaultdict(list)
    xgm.eval()
    phases = ["train", "valid"]
    for phase in phases:
        intermediate_output = []
        labels_output = []
        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
            images = images.to(device)
            labels = labels.to(device)
            intermediate_output.append(xgm(images).squeeze())
            labels_output.append(labels)
        xgb_data[phase].append(torch.cat(intermediate_output).cpu().numpy())
        xgb_data[phase].append(torch.cat(labels_output).cpu().numpy())
    
    fold_list.append(xgb_data)

In [160]:
print(gridcv.best_params_)

{'colsample_bytree': 0.3, 'max_depth': 6, 'min_child_weight': 1}


In [165]:
print(gridcv.best_params_)

{'colsample_bytree': 0.1}


In [169]:
from sklearn.model_selection import GridSearchCV

# 객체 생성, 일단은 트리 100개만 만듦
xgb_model = XGBClassifier(min_child_weight = 1, max_depth=6, colsample_bytree = 0.1, n_estimators=200)

# 후보 파라미터 선정
params = {'learning_rate':[0.1, 0.2, 0.3]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv = GridSearchCV(xgb_model, param_grid=params, cv=3)

# 파라미터 튜닝 시작
gridcv.fit(X_reduced, xgb_data['train'][1], early_stopping_rounds=30, eval_metric='mlogloss', eval_set=[(X_valid_reduced, y_true)],
           verbose=100)

#튜닝된 파라미터 출력
print(gridcv.best_params_)

[0]	validation_0-mlogloss:2.45864
[100]	validation_0-mlogloss:0.07397
[199]	validation_0-mlogloss:0.05098
[0]	validation_0-mlogloss:2.46183
[100]	validation_0-mlogloss:0.07580
[199]	validation_0-mlogloss:0.05193
[0]	validation_0-mlogloss:2.45708
[100]	validation_0-mlogloss:0.07411
[199]	validation_0-mlogloss:0.05107
[0]	validation_0-mlogloss:2.06922
[100]	validation_0-mlogloss:0.05498
[199]	validation_0-mlogloss:0.05055
[0]	validation_0-mlogloss:2.07470
[100]	validation_0-mlogloss:0.05491
[199]	validation_0-mlogloss:0.05062
[0]	validation_0-mlogloss:2.06590
[100]	validation_0-mlogloss:0.05392
[199]	validation_0-mlogloss:0.04962
[0]	validation_0-mlogloss:1.74463
[100]	validation_0-mlogloss:0.05316
[199]	validation_0-mlogloss:0.05123
[0]	validation_0-mlogloss:1.75069
[100]	validation_0-mlogloss:0.05554
[199]	validation_0-mlogloss:0.05357
[0]	validation_0-mlogloss:1.73902
[100]	validation_0-mlogloss:0.05271
[199]	validation_0-mlogloss:0.05066
[0]	validation_0-mlogloss:1.72097
[100]	valida

### 결과

In [170]:
# 1차적으로 튜닝된 파라미터를 가지고 객체 생성
xgb_model = XGBClassifier(max_depth=6, min_child_weight=1, colsample_bytree=0.1, n_estimators=200)

# 학습
xgb_model.fit(X_reduced, xgb_data['train'][1], early_stopping_rounds=30, eval_metric='mlogloss', eval_set=[(X_valid_reduced, y_true)],
             verbose=50)

[0]	validation_0-mlogloss:1.72097
[50]	validation_0-mlogloss:0.05703
[100]	validation_0-mlogloss:0.04995
[150]	validation_0-mlogloss:0.04862
[199]	validation_0-mlogloss:0.04837


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [171]:
y_pred_reduced = xgb_model.predict(X_valid_reduced)
y_true = xgb_data['valid'][1]

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

print(f'Accuracy: {accuracy_score(y_true, y_pred_reduced)}, f1_score: {f1_score(y_true, y_pred_reduced, average="macro")}')

Accuracy: 0.9899364406779662, f1_score: 0.9786035649032008


# Catboost 학습

In [173]:
from catboost import CatBoostClassifier, Pool

In [174]:
train_data = Pool(data=X_reduced, label=xgb_data['train'][1])
valid_data = Pool(data=X_valid_reduced, label=xgb_data['valid'][1])

model_cat = CatBoostClassifier()
model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)

Learning rate set to 0.114004
0:	learn: 1.5117918	test: 1.5549815	best: 1.5549815 (0)	total: 193ms	remaining: 3m 12s
100:	learn: 0.0084193	test: 0.0459876	best: 0.0459520 (98)	total: 12.3s	remaining: 1m 49s
200:	learn: 0.0044674	test: 0.0424127	best: 0.0423932 (199)	total: 24.4s	remaining: 1m 36s
300:	learn: 0.0028699	test: 0.0416555	best: 0.0416100 (292)	total: 36.7s	remaining: 1m 25s
400:	learn: 0.0020153	test: 0.0413092	best: 0.0412753 (389)	total: 48.8s	remaining: 1m 12s
500:	learn: 0.0015395	test: 0.0410304	best: 0.0410130 (484)	total: 1m	remaining: 1m
600:	learn: 0.0012108	test: 0.0409734	best: 0.0408020 (562)	total: 1m 13s	remaining: 48.5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.04080204801
bestIteration = 562

Shrink model to first 563 iterations.


In [175]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

y_pred_reduced = model_cat.predict(X_valid_reduced)
y_true = xgb_data['valid'][1]

print(f'Accuracy: {accuracy_score(y_true, y_pred_reduced)}, f1_score: {f1_score(y_true, y_pred_reduced, average="macro")}')

Accuracy: 0.9909957627118644, f1_score: 0.9830412027135461


In [178]:
y_pred_reduced.reshape(-1)

array([4, 4, 4, ..., 6, 0, 0])

In [177]:
y_true

array([4, 4, 4, ..., 6, 0, 0])

In [179]:
test_data = []
intermediate_output = []
for ind, images in enumerate(tqdm(loader)):
    images = images.to(device)
    intermediate_output.append(xgm(images).squeeze())
test_data.append(torch.cat(intermediate_output).cpu().numpy())

In [180]:
test_data[0].shape

(12600, 1792)

In [183]:
test_reduced = pca.transform(test_data[0])

In [184]:
test_reduced

array([[ 3.8537211 ,  1.2231947 , -4.1451263 , ..., -0.17371412,
        -0.16318451,  0.11631411],
       [ 1.2126782 , -4.1606083 , -0.80252934, ..., -0.16583988,
         0.30784526, -0.1896185 ],
       [ 3.338316  ,  1.8520613 , -5.008249  , ..., -0.23278433,
        -0.2318383 , -0.18566856],
       ...,
       [ 3.033539  ,  9.750283  ,  5.481638  , ..., -0.08406852,
         0.14275274, -0.16424063],
       [-0.7741766 , -7.1764    , -3.0245001 , ..., -0.32851264,
        -0.42317456,  0.0351516 ],
       [ 3.6438801 ,  6.477304  ,  6.2143836 , ...,  0.03459899,
        -0.05342256, -0.16162072]], dtype=float32)

In [185]:
y_pred = model_cat.predict(test_reduced)

In [186]:
y_pred

array([[13],
       [ 1],
       [13],
       ...,
       [ 9],
       [ 1],
       [ 7]])

# Catboost 5-fold

In [225]:
# Custom Model Template
class b4_0_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b4 = model_b4[0]
        
    def forward(self, x):
        x = self.b4.conv_stem(x)
        x = self.b4.bn1(x)
        x = self.b4.act1(x)
        x = self.b4.blocks(x)
        x = self.b4.conv_head(x)
        x = self.b4.bn2(x)
        x = self.b4.act2(x)
        x = self.b4.global_pool(x)
        return x
    
class b4_1_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b4 = model_b4[1]
        
    def forward(self, x):
        x = self.b4.conv_stem(x)
        x = self.b4.bn1(x)
        x = self.b4.act1(x)
        x = self.b4.blocks(x)
        x = self.b4.conv_head(x)
        x = self.b4.bn2(x)
        x = self.b4.act2(x)
        x = self.b4.global_pool(x)
        return x
    
class b4_2_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b4 = model_b4[2]
        
    def forward(self, x):
        x = self.b4.conv_stem(x)
        x = self.b4.bn1(x)
        x = self.b4.act1(x)
        x = self.b4.blocks(x)
        x = self.b4.conv_head(x)
        x = self.b4.bn2(x)
        x = self.b4.act2(x)
        x = self.b4.global_pool(x)
        return x

class b4_3_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b4 = model_b4[3]
        
    def forward(self, x):
        x = self.b4.conv_stem(x)
        x = self.b4.bn1(x)
        x = self.b4.act1(x)
        x = self.b4.blocks(x)
        x = self.b4.conv_head(x)
        x = self.b4.bn2(x)
        x = self.b4.act2(x)
        x = self.b4.global_pool(x)
        return x

class b4_4_for_xgboost(nn.Module):
    def __init__(self):
        super().__init__()
        self.b4 = model_b4[4]
        
    def forward(self, x):
        x = self.b4.conv_stem(x)
        x = self.b4.bn1(x)
        x = self.b4.act1(x)
        x = self.b4.blocks(x)
        x = self.b4.conv_head(x)
        x = self.b4.bn2(x)
        x = self.b4.act2(x)
        x = self.b4.global_pool(x)
        return x

In [226]:
device = "cuda" if torch.cuda.is_available() else "cpu"
xgm_0 = b4_0_for_xgboost().to(device)
xgm_1 = b4_1_for_xgboost().to(device)
xgm_2 = b4_2_for_xgboost().to(device)
xgm_3 = b4_3_for_xgboost().to(device)
xgm_4 = b4_4_for_xgboost().to(device)

for params in xgm_0.parameters():
    params.requires_grad = False
for params in xgm_1.parameters():
    params.requires_grad = False
for params in xgm_2.parameters():
    params.requires_grad = False
for params in xgm_3.parameters():
    params.requires_grad = False
for params in xgm_4.parameters():
    params.requires_grad = False

xgm = [xgm_0, xgm_1, xgm_2, xgm_3, xgm_4]

# 동일한 기본값 설정
data_dict = {}
for idx, fold in tqdm(enumerate(folds)):
    train_data = CustomDatasetFromDF(df.iloc[fold[0]])
    valid_data = CustomDatasetFromDF(df.iloc[fold[1]])

    BATCH_SIZE = 16
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,
                                                   num_workers=4, drop_last=True)
    valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False,
                                                   num_workers=4, drop_last=True)

    dataloaders = {"train" : train_dataloader, "valid" : valid_dataloader}

    from collections import defaultdict

    xgb_data = defaultdict(list)
    xgm[idx].eval()
    phases = ["train", "valid"]
    for phase in phases:
        intermediate_output = []
        labels_output = []
        for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
            images = images.to(device)
            labels = labels.to(device)
            intermediate_output.append(xgm[idx](images).squeeze())
            labels_output.append(labels)
        xgb_data[phase].append(torch.cat(intermediate_output).cpu().numpy())
        xgb_data[phase].append(torch.cat(labels_output).cpu().numpy())
    
    data_dict[idx] = xgb_data

In [227]:
data = data_dict[1]
data['valid'][0].shape, data['valid'][1].shape

((3776, 1792), (3776,))

In [228]:
data['train'][0].shape, data['train'][1].shape

((15120, 1792), (15120,))

In [229]:
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

cat_models = {}
pca_models = {}
for i in range(5):
    print(f'----------------------fold: {i+1}----------------------')
    data = data_dict[i]
    X_train, y_train, X_val, y_val = data['train'][0], data['train'][1], data['valid'][0], data['valid'][1]

    pca = PCA(n_components=0.95)
    X_train_reduced = pca.fit_transform(X_train)
    X_val_reduced = pca.transform(X_val)
    
    train_data = Pool(data=X_train_reduced, label=y_train)
    valid_data = Pool(data=X_val_reduced, label=y_val)

    model_cat = CatBoostClassifier()
    model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
    
    cat_models[i] = model_cat
    pca_models[i] = pca
    
    y_pred_val_reduced = model_cat.predict(X_val_reduced)
    y_true_val = y_val

    print(f'Accuracy: {accuracy_score(y_true_val, y_pred_val_reduced)}, f1_score: {f1_score(y_true_val, y_pred_val_reduced, average="macro")}')
    print(f'------------------fold: {i+1} end----------------------')

----------------------fold: 1----------------------
Learning rate set to 0.114004
0:	learn: 1.6382566	test: 1.6793519	best: 1.6793519 (0)	total: 159ms	remaining: 2m 38s
100:	learn: 0.0155668	test: 0.0460882	best: 0.0460882 (100)	total: 15.8s	remaining: 2m 21s
200:	learn: 0.0082529	test: 0.0396422	best: 0.0396416 (199)	total: 31.4s	remaining: 2m 4s
300:	learn: 0.0052542	test: 0.0363801	best: 0.0363698 (289)	total: 47.3s	remaining: 1m 49s
400:	learn: 0.0038336	test: 0.0348665	best: 0.0348357 (399)	total: 1m 2s	remaining: 1m 33s
500:	learn: 0.0029222	test: 0.0333238	best: 0.0333238 (500)	total: 1m 18s	remaining: 1m 17s
600:	learn: 0.0022382	test: 0.0320939	best: 0.0320932 (598)	total: 1m 34s	remaining: 1m 2s
700:	learn: 0.0018047	test: 0.0314495	best: 0.0314495 (700)	total: 1m 49s	remaining: 46.9s
800:	learn: 0.0015104	test: 0.0311174	best: 0.0311174 (800)	total: 2m 5s	remaining: 31.2s
900:	learn: 0.0013020	test: 0.0308861	best: 0.0308861 (900)	total: 2m 20s	remaining: 15.5s
999:	learn: 0

## 테스트 데이터

In [272]:
test_dict = {}
xgm = [xgm_0, xgm_1, xgm_2, xgm_3, xgm_4]

for idx in range(5):
    intermediate_output = []
    for images in tqdm(loader):
        images = images.to(device)
        intermediate_output.append(xgm[idx](images).squeeze())
    test_dict[idx] = torch.cat(intermediate_output).cpu().numpy()

In [276]:
test_dict[0].shape

(12600, 1792)

In [277]:
X_0_test_reduced = pca_models[0].transform(test_dict[0])
X_1_test_reduced = pca_models[1].transform(test_dict[1])
X_2_test_reduced = pca_models[2].transform(test_dict[2])
X_3_test_reduced = pca_models[3].transform(test_dict[3])
X_4_test_reduced = pca_models[4].transform(test_dict[4])

X_test_reduceds = [X_0_test_reduced, X_1_test_reduced, X_2_test_reduced, X_3_test_reduced, X_4_test_reduced]

In [278]:
proba = 0
for idx, X_test_reduced in enumerate(X_test_reduceds):
    proba += cat_models[idx].predict_proba(X_test_reduced)

proba.shape

(12600, 18)

In [279]:
y_pred = np.argmax(proba, axis=-1)

In [280]:
y_pred

array([13,  1, 13, ...,  9,  1,  7])

In [281]:
submission['ans'] = y_pred

In [282]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1


In [283]:
submission.to_csv(os.path.join(test_dir, 'catboost_5fold.csv'), index=False)
print('test inference is done!')

test inference is done!


In [323]:
# GPU를 활용해 학습하면 훨씬 빠릅니다.
# Catboost 공식 문서를 읽고 알게 되었습니다.

from sklearn.decomposition import PCA
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

cat_models = {}
pca_models = {}
for i in range(5):
    print(f'----------------------fold: {i+1}----------------------')
    data = data_dict[i]
    X_train, y_train, X_val, y_val = data['train'][0], data['train'][1], data['valid'][0], data['valid'][1]

    pca = PCA(n_components=0.95)
    X_train_reduced = pca.fit_transform(X_train)
    X_val_reduced = pca.transform(X_val)
    
    train_data = Pool(data=X_train_reduced, label=y_train)
    valid_data = Pool(data=X_val_reduced, label=y_val)

    model_cat = CatBoostClassifier(task_type="GPU", devices='cuda')
    model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)
    
    cat_models[i] = model_cat
    pca_models[i] = pca
    
    y_pred_val_reduced = model_cat.predict(X_val_reduced)
    y_true_val = y_val

    print(f'Accuracy: {accuracy_score(y_true_val, y_pred_val_reduced)}, f1_score: {f1_score(y_true_val, y_pred_val_reduced, average="macro")}')
    print(f'------------------fold: {i+1} end----------------------')

----------------------fold: 1----------------------
Learning rate set to 0.138346
0:	learn: 1.4783275	test: 1.5225362	best: 1.5225362 (0)	total: 11.3ms	remaining: 11.2s


100:	learn: 0.0134080	test: 0.0455767	best: 0.0455767 (100)	total: 875ms	remaining: 7.79s
200:	learn: 0.0061191	test: 0.0381196	best: 0.0381196 (200)	total: 1.69s	remaining: 6.71s
300:	learn: 0.0036214	test: 0.0355943	best: 0.0355885 (297)	total: 2.71s	remaining: 6.28s
400:	learn: 0.0023964	test: 0.0343317	best: 0.0342468 (393)	total: 3.53s	remaining: 5.28s
500:	learn: 0.0018028	test: 0.0334310	best: 0.0334292 (498)	total: 4.33s	remaining: 4.31s
600:	learn: 0.0014154	test: 0.0327572	best: 0.0327435 (596)	total: 5.18s	remaining: 3.44s
700:	learn: 0.0011354	test: 0.0324871	best: 0.0324871 (700)	total: 5.98s	remaining: 2.55s
800:	learn: 0.0009516	test: 0.0321992	best: 0.0321978 (797)	total: 6.77s	remaining: 1.68s
900:	learn: 0.0008279	test: 0.0321290	best: 0.0321110 (884)	total: 7.62s	remaining: 837ms
999:	learn: 0.0007270	test: 0.0321209	best: 0.0321086 (905)	total: 8.41s	remaining: 0us
bestTest = 0.03210859784
bestIteration = 905
Shrink model to first 906 iterations.
Accuracy: 0.9920550

100:	learn: 0.0077633	test: 0.0500077	best: 0.0500077 (100)	total: 984ms	remaining: 8.76s
200:	learn: 0.0034253	test: 0.0440317	best: 0.0440317 (200)	total: 1.95s	remaining: 7.76s
300:	learn: 0.0020483	test: 0.0424398	best: 0.0424279 (296)	total: 2.87s	remaining: 6.66s
400:	learn: 0.0014514	test: 0.0418665	best: 0.0418665 (400)	total: 3.62s	remaining: 5.41s
500:	learn: 0.0011119	test: 0.0415318	best: 0.0415094 (499)	total: 4.37s	remaining: 4.35s
600:	learn: 0.0008842	test: 0.0414240	best: 0.0413734 (564)	total: 5.12s	remaining: 3.4s
700:	learn: 0.0007388	test: 0.0412879	best: 0.0412879 (700)	total: 5.87s	remaining: 2.5s
800:	learn: 0.0006448	test: 0.0412294	best: 0.0412294 (800)	total: 6.62s	remaining: 1.64s
900:	learn: 0.0005596	test: 0.0411547	best: 0.0411220 (865)	total: 7.37s	remaining: 809ms
bestTest = 0.04112195969
bestIteration = 865
Shrink model to first 866 iterations.
Accuracy: 0.9907309322033898, f1_score: 0.975634801888402
------------------fold: 2 end----------------------

100:	learn: 0.0058374	test: 0.0405464	best: 0.0405464 (100)	total: 771ms	remaining: 6.86s
200:	learn: 0.0027108	test: 0.0375507	best: 0.0375397 (196)	total: 1.49s	remaining: 5.92s
300:	learn: 0.0016310	test: 0.0364860	best: 0.0364584 (292)	total: 2.22s	remaining: 5.15s
400:	learn: 0.0011689	test: 0.0362639	best: 0.0362059 (345)	total: 2.94s	remaining: 4.39s
bestTest = 0.03620592618
bestIteration = 345
Shrink model to first 346 iterations.
Accuracy: 0.991260593220339, f1_score: 0.9790711014909443
------------------fold: 3 end----------------------
----------------------fold: 4----------------------
Learning rate set to 0.138346
0:	learn: 1.2195665	test: 1.2671440	best: 1.2671440 (0)	total: 9.25ms	remaining: 9.24s


100:	learn: 0.0074003	test: 0.0505132	best: 0.0505132 (100)	total: 756ms	remaining: 6.72s
200:	learn: 0.0035487	test: 0.0462194	best: 0.0461307 (195)	total: 1.46s	remaining: 5.79s
300:	learn: 0.0020296	test: 0.0446944	best: 0.0446944 (300)	total: 2.17s	remaining: 5.03s
400:	learn: 0.0013046	test: 0.0444952	best: 0.0444032 (376)	total: 2.88s	remaining: 4.31s
500:	learn: 0.0009938	test: 0.0444647	best: 0.0443708 (432)	total: 3.67s	remaining: 3.65s
bestTest = 0.04437084925
bestIteration = 432
Shrink model to first 433 iterations.
Accuracy: 0.9902012711864406, f1_score: 0.9809589406518737
------------------fold: 4 end----------------------
----------------------fold: 5----------------------
Learning rate set to 0.138346
0:	learn: 1.3546639	test: 1.4179293	best: 1.4179293 (0)	total: 8.96ms	remaining: 8.95s


100:	learn: 0.0060172	test: 0.0420039	best: 0.0420039 (100)	total: 727ms	remaining: 6.47s
200:	learn: 0.0023857	test: 0.0355274	best: 0.0355274 (200)	total: 1.41s	remaining: 5.61s
300:	learn: 0.0014417	test: 0.0336721	best: 0.0336721 (300)	total: 2.07s	remaining: 4.82s
400:	learn: 0.0010448	test: 0.0326368	best: 0.0326285 (399)	total: 2.75s	remaining: 4.11s
500:	learn: 0.0007988	test: 0.0322904	best: 0.0322504 (494)	total: 3.41s	remaining: 3.4s
600:	learn: 0.0006512	test: 0.0319004	best: 0.0318901 (598)	total: 4.07s	remaining: 2.7s
700:	learn: 0.0005603	test: 0.0315029	best: 0.0315029 (700)	total: 4.73s	remaining: 2.02s
800:	learn: 0.0004793	test: 0.0313967	best: 0.0313757 (783)	total: 5.4s	remaining: 1.34s
bestTest = 0.03137565265
bestIteration = 783
Shrink model to first 784 iterations.
Accuracy: 0.991260593220339, f1_score: 0.9795349089864369
------------------fold: 5 end----------------------
